### loading modules and data

In [1]:
from models import *
from utils import *
from data_utils import *

dataset_path = './data/'
X_train_valid = np.load(dataset_path + "X_train_valid.npy")
y_train_valid = np.load(dataset_path + "y_train_valid.npy")
person_train_valid = np.load(dataset_path + "person_train_valid.npy")
X_test = np.load(dataset_path + "X_test.npy")
y_test = np.load(dataset_path + "y_test.npy")
person_test = np.load(dataset_path + "person_test.npy")
print ('Training/Valid data shape: {}'.format(X_train_valid.shape))
print ('Test data shape: {}'.format(X_test.shape))
print ('Training/Valid target shape: {}'.format(y_train_valid.shape))
print ('Test target shape: {}'.format(y_test.shape))
print ('Person train/valid shape: {}'.format(person_train_valid.shape))
print ('Person test shape: {}'.format(person_test.shape))

train_fold, val_fold = Train_Val_Data(X_train_valid, y_train_valid)



GPU is available
Training/Valid data shape: (2115, 22, 1000)
Test data shape: (443, 22, 1000)
Training/Valid target shape: (2115,)
Test target shape: (443,)
Person train/valid shape: (2115, 1)
Person test shape: (443, 1)


### K-Fold Training and Validation (use k=1 to get results faster)

In [6]:
# setting up the data augmentation here
aug_type = "window"
window_size = 200
vote_num = 20

for k in range(1):
    # indicate hyperparameters here
    model, criterion, optimizer = InitRNN(rnn_type='CNNGRU', dropout=0.4)
    print ('fold {}'.format(k+1))
    X_train, y_train, p_train = X_train_valid[train_fold[k]], y_train_valid[train_fold[k]], person_train_valid[train_fold[k]]
    X_val, y_val, p_val = X_train_valid[val_fold[k]], y_train_valid[val_fold[k]], person_train_valid[val_fold[k]]
    mu = np.mean(X_train, axis=(0,2), keepdims = True) 
    std = np.std(X_train, axis=(0,2), keepdims = True)
    X_train = standardize(X_train, mu, std)
    X_val = standardize(X_val, mu, std)
    X_train, y_train, p_train = Aug_Data(X_train, y_train, p_train, aug_type=aug_type, window_size=window_size)
    if aug_type != 'window':
        X_val, y_val, p_val = Aug_Data(X_val, y_val, p_val, aug_type=aug_type)
    EEG_trainset = EEG_Dataset(X_train=X_train, y_train=y_train, p_train=p_train, mode='train')
    EEG_trainloader = DataLoader(EEG_trainset, batch_size=128, shuffle=True)
    EEG_valset = EEG_Dataset(X_val=X_val, y_val=y_val, p_val=p_val, mode='val')
    EEG_valloader = DataLoader(EEG_valset, batch_size=128, shuffle=False)
    best_model = TrainValRNN(model, criterion, optimizer, EEG_trainloader, EEG_valloader, num_epochs=20, aug_type=aug_type, window_size=window_size, vote_num=vote_num)


fold 1
epoch: 1      time: 16.18    loss: 730.066    train acc: 0.279    val acc: 0.326
saving best model...
epoch: 2      time: 16.74    loss: 722.552    train acc: 0.307    val acc: 0.369
saving best model...
epoch: 3      time: 16.67    loss: 708.840    train acc: 0.343    val acc: 0.414
saving best model...
epoch: 4      time: 16.13    loss: 690.412    train acc: 0.378    val acc: 0.435
saving best model...
epoch: 5      time: 16.00    loss: 677.129    train acc: 0.397    val acc: 0.423
epoch: 6      time: 16.28    loss: 665.648    train acc: 0.417    val acc: 0.435
saving best model...
epoch: 7      time: 16.44    loss: 654.787    train acc: 0.430    val acc: 0.430
epoch: 8      time: 16.17    loss: 649.136    train acc: 0.439    val acc: 0.449
saving best model...
epoch: 9      time: 16.14    loss: 639.905    train acc: 0.451    val acc: 0.437
epoch: 10     time: 15.11    loss: 633.710    train acc: 0.458    val acc: 0.421
epoch: 11     time: 15.38    loss: 625.696    train acc: 

### Training and Testing

In [8]:
X_test = standardize(X_test, mu, std)
TestRNN(best_model, X_test, y_test, person_test, aug_type=aug_type, window_size=window_size, vote_num=vote_num)

Testing Accuracy: 0.4763
